<a href="https://colab.research.google.com/github/eangelb29/NGG6050_Assignments/blob/main/ConfidenceIntervals_AB91025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

# Settings
np.random.seed(42)
mu = 10          # True population mean
sigma = 2        # True population standard deviation
sample_sizes = [5, 10, 20, 40, 80, 160, 1000]
confidence_level = 0.95
z_score = stats.norm.ppf(0.5 + confidence_level / 2)

# Function to compute Bayesian credible interval assuming conjugate prior N(0, 100^2)
def bayesian_credible_interval(sample, prior_mu=0, prior_sigma=100):
    n = len(sample)
    sample_mean = np.mean(sample)
    sample_var = np.var(sample, ddof=1)

    # Posterior parameters
    posterior_variance = 1 / (n / sample_var + 1 / prior_sigma**2)
    posterior_mean = posterior_variance * (sample_mean * n / sample_var + prior_mu / prior_sigma**2)
    posterior_std = np.sqrt(posterior_variance)

    ci_lower = stats.norm.ppf(0.025, loc=posterior_mean, scale=posterior_std)
    ci_upper = stats.norm.ppf(0.975, loc=posterior_mean, scale=posterior_std)

    return ci_lower, ci_upper

# Storage for results
results = []

# Main loop
for n in sample_sizes:
    sample = np.random.normal(loc=mu, scale=sigma, size=n)
    sample_mean = np.mean(sample)
    sample_std = np.std(sample, ddof=1)

    # 1. Simple analytic (known sigma)
    margin_z = z_score * sigma / np.sqrt(n)
    ci_known_sigma = (sample_mean - margin_z, sample_mean + margin_z)

    # 2. Simple analytic (unknown sigma, small n)
    t_score = stats.t.ppf(0.5 + confidence_level / 2, df=n-1)
    margin_t = t_score * sample_std / np.sqrt(n)
    ci_unknown_sigma = (sample_mean - margin_t, sample_mean + margin_t)

    # 3. Bootstrap confidence interval
    n_bootstrap = 10000
    bootstrap_means = [np.mean(np.random.choice(sample, size=n, replace=True)) for _ in range(n_bootstrap)]
    ci_bootstrap = (np.percentile(bootstrap_means, 2.5), np.percentile(bootstrap_means, 97.5))

    # 4. Bayesian credible interval
    ci_bayesian = bayesian_credible_interval(sample)

    results.append({
        'n': n,
        'sample_mean': sample_mean,
        'CI_known_sigma': ci_known_sigma,
        'CI_unknown_sigma': ci_unknown_sigma,
        'CI_bootstrap': ci_bootstrap,
        'CI_bayesian': ci_bayesian
    })

# Display results
for res in results:
    print(f"n = {res['n']}")
    print(f"  Sample Mean: {res['sample_mean']:.4f}")
    print(f"  CI (Known σ): {res['CI_known_sigma']}")
    print(f"  CI (Unknown σ, t-distribution): {res['CI_unknown_sigma']}")
    print(f"  CI (Bootstrap): {res['CI_bootstrap']}")
    print(f"  CI (Bayesian): {res['CI_bayesian']}")
    print()

n = 5
  Sample Mean: 10.9180
  CI (Known σ): (np.float64(9.16496086749701), np.float64(12.671051029803335))
  CI (Unknown σ, t-distribution): (np.float64(9.15923305157548), np.float64(12.676778845724865))
  CI (Bootstrap): (np.float64(9.900751520488877), np.float64(12.031405522461434))
  CI (Bayesian): (np.float64(9.676029756724168), np.float64(12.159105952694981))

n = 10
  Sample Mean: 10.3340
  CI (Known σ): (np.float64(9.094449361905331), np.float64(11.573629491123578))
  CI (Unknown σ, t-distribution): (np.float64(9.290791538589884), np.float64(11.377287314439025))
  CI (Bootstrap): (np.float64(9.5358917022701), np.float64(11.250350232638855))
  CI (Bayesian): (np.float64(9.429944982134394), np.float64(11.237694308679423))

n = 20
  Sample Mean: 10.5122
  CI (Known σ): (np.float64(9.63572727971144), np.float64(11.388772360864605))
  CI (Unknown σ, t-distribution): (np.float64(9.761026538129356), np.float64(11.26347310244669))
  CI (Bootstrap): (np.float64(9.86323878730902), np.flo